# MongoDB Atlas

>[MongoDB Atlas](https://www.mongodb.com/) 是一个文档数据库，可用作向量数据库。

在本指南中，我们将演示带 `MongoDB Atlas` 向量存储的 `SelfQueryRetriever`。

## 创建 MongoDB Atlas 向量数据库
首先，我们将创建一个 MongoDB Atlas 向量数据库并填充一些数据。我们创建了一个包含电影摘要的小型演示文档集。

注意：自查询检索器要求您安装 `lark`（`pip install lark`）。我们还需要 `pymongo` 包。

In [5]:
%pip install --upgrade --quiet  lark pymongo

我们想要使用 `OpenAIEmbeddings`，所以我们需要获取 OpenAI API 密钥。

In [ ]:
import os

OPENAI_API_KEY = "Use your OpenAI key"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from pymongo import MongoClient

CONNECTION_STRING = "Use your MongoDB Atlas connection string"
DB_NAME = "Name of your MongoDB Atlas database"
COLLECTION_NAME = "Name of your collection in the database"
INDEX_NAME = "Name of a search index defined on the collection"

MongoClient = MongoClient(CONNECTION_STRING)
collection = MongoClient[DB_NAME][COLLECTION_NAME]

embeddings = OpenAIEmbeddings()

In [ ]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "action"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "genre": "thriller", "rating": 8.2},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "rating": 8.3, "genre": "drama"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={"year": 1979, "rating": 9.9, "genre": "science fiction"},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "genre": "thriller", "rating": 9.0},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated", "rating": 9.3},
    ),
]

vectorstore = MongoDBAtlasVectorSearch.from_documents(
    docs,
    embeddings,
    collection=collection,
    index_name=INDEX_NAME,
)

现在，让我们在集群上创建一个向量搜索索引。在下面的示例中，`embedding` 是包含嵌入向量的字段的名称。请参阅[文档](https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector)以获取有关如何定义 Atlas Vector Search 索引的更多详细信息。
您可以将索引命名为 `{COLLECTION_NAME}`，并在命名空间 `{DB_NAME}.{COLLECTION_NAME}` 上创建索引。最后，在 MongoDB Atlas 的 JSON 编辑器中写入以下定义：

```json
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "dimensions": 1536,
        "similarity": "cosine",
        "type": "knnVector"
      },
      "genre": {
        "type": "token"
      },
      "ratings": {
        "type": "number"
      },
      "year": {
        "type": "number"
      }
    }
  }
}
```

## 创建我们自查询的检索器
现在我们可以实例化我们的检索器了。要做到这一点，我们需要提前提供一些有关我们的文档所支持的元数据字段以及文档内容的简短描述的信息。

In [ ]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"

In [ ]:
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## 测试一下
现在我们可以试试实际使用我们的检索器了！

In [ ]:
# This example only specifies a relevant query
retriever.invoke("What are some movies about dinosaurs")

In [ ]:
# This example specifies a filter
retriever.invoke("What are some highly rated movies (above 9)?")

In [ ]:
# This example only specifies a query and a filter
retriever.invoke("I want to watch a movie about toys rated higher than 9")

In [ ]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above or equal 9) thriller film?")

In [ ]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about dinosaurs, \
    and preferably has a lot of action"
)

## 过滤 k

我们还可以使用自查询检索器来指定 `k`：要获取的文档数量。

我们可以通过向构造函数传递 `enable_limit=True` 来实现此目的。

In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True,
    enable_limit=True,
)

In [ ]:
# This example only specifies a relevant query
retriever.invoke("What are two movies about dinosaurs?")